In [20]:
import pandas as pd
import numpy as np

# import file
df = pd.read_csv('../../res/tp2/german_credit.csv', sep=',') # columns: admit, gre, gpa, rank; 399 rows

continuous_attrs = {'Duration of Credit (month)': ['<= 12', '<= 24', '<= 36', '<= 48', '<= 60', '> 60'],
                    'Credit Amount': ['<= 500', '<= 1000', '<= 1500', '<= 2000', '<= 2500', '<= 3000', '<= 3500', '<= 4000', 
                                      '<= 4500', '<= 5000', '<= 5500', '<= 6000', '<= 6500', '<= 7000', '<= 7500', '<= 8000',
                                      '<= 9000', '<= 10000', '<= 11000', '<= 12000', '<= 13000', '<= 14000', '<= 15000', '> 15000'],
                    'Age (years)': ['< 30', '< 40', '< 50', '< 60', '>= 60']}

In [21]:
from sklearn.model_selection import train_test_split

# Separar los datos en datos de entrenamiento y testeo
train, test = train_test_split(df, test_size=0.2)
train.head()
test.head()

,Creditability,Account Balance,Duration of Credit (month),Payment Status of Previous Credit,Purpose,Credit Amount,Value Savings/Stocks,Length of current employment,Instalment per cent,Sex & Marital Status,...,Duration in Current address,Most valuable available asset,Age (years),Concurrent Credits,Type of apartment,No of Credits at this Bank,Occupation,No of dependents,Telephone,Foreign Worker
652,1,2,12,2,8,754,5,5,4,3,...,4,2,38,3,2,2,3,1,1,1
600,0,1,36,2,1,8229,1,3,2,3,...,2,2,26,3,2,1,3,2,1,1
632,1,1,24,2,2,4169,1,3,4,3,...,4,2,28,3,2,1,3,1,1,1
464,1,2,12,2,0,7472,5,1,1,2,...,2,1,24,3,1,1,1,1,1,1
432,1,2,15,4,9,2326,3,3,2,3,...,4,3,27,1,2,1,3,1,1,1


In [25]:
import math

def get_entropy_term(p):
    return - p * math.log2(p) if not math.isclose(p, 0) else 0

entropy = None

def get_entropy(data, cond): # H(S)
    if entropy is not None:
        return entropy
    
    positive_freq = len(data[cond]) / len(data)
    negative_freq = 1 - positive_freq
    entropy = get_entropy_term(positive_freq) + get_entropy_term(negative_freq)
    
    return entropy

def get_attr_entropy(data, cond): # H(Sv)
    positive_freq = len(data[cond]) / len(data)
    negative_freq = 1 - positive_freq
    return get_entropy_term(positive_freq) + get_entropy_term(negative_freq)

def get_rows_by_attr_cond(data, attr, cond, cond_index):
    rows = None
    
    if i == 0:
        rows = data.query(f'{attr} {cond}')
    else:
        prev_cond = continuous_attrs[attr][i-1]
        rows = data.query(f'not({attr} {prev_cond}) and {attr} {cond}')
        
    return rows
    
def gain(data, attr, class_cond):
    ret = entropy
    
    if attr in continuous_attrs:
        for i, cond in enumerate(continuous_attrs[attr]):
            rows = get_rows_by_attr_cond(data, attr, cond, i)
            ret -= len(rows) / len(data) * get_attr_entropy(rows, class_cond)
    else:
        for value in df[attr].unique():
            rows = data.query(f'{attr} == {value}')
            print(get_attr_entropy(rows, class_cond))
            ret -= len(rows) / len(data) * get_attr_entropy(rows, class_cond)
    
    return ret

gains = {}
for attr in train.columns:
    gains[attr] = gain(train, attr, train.Creditability == 1)
print(gains)
    
#df['Credit Amount'].hist(grid=True, bins=30, rwidth=0.9, color='#607c8e')
#print(get_entropy(train, train.Creditability == 1))

0.0


<ipython-input-25-acc2c4d380bf>:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  positive_freq = len(data[cond]) / len(data)


TypeError: unsupported operand type(s) for -=: 'NoneType' and 'float'